In [ ]:
# Installing dependencies
!pip install pandas matplotlib seaborn numpy

In [2]:
#Importing Libraries 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np

In [4]:
#reading csv files for data-cleaning
df_sales = pd.read_csv('Sales Transactions.csv')
df_customers = pd.read_csv('Customer Information.csv') 
df_region = pd.read_csv('Region Metadata.csv')

In [ ]:
# Filling missing values using median
df_sales['ContractLength'].fillna(df_sales['ContractLength'].median(), inplace=True)

In [28]:
# Using IQR MethodDetect outliers in revenue 
def detect_outlier(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3-Q1
    lower = Q1-1.5*IQR
    higher = Q3+1.5*IQR 
    return series[(series<lower)|(series>higher)] 
    

In [31]:
#Declaring outliers to  a variable
revenue_outliers = detect_outlier(df_sales['Revenue'] )


In [32]:
#Printing Outliers
print({"Revenue_Outlier_Count": len(revenue_outliers)})


{'Revenue_Outlier_Count': 493}


In [33]:
# remove outliers in revenue by indeces 
def detect_outlier_indices(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3-Q1
    lower = Q1-1.5*IQR
    higher = Q3+1.5*IQR 
    return series[(series<lower)|(series>higher)].index 

In [34]:
#Assigning removed outliers to a variable
revenue_outliers_indecies = detect_outlier_indices(df_sales['Revenue'])

In [35]:
#Dropping Outliers from the dataset
df_clean_sales = df_sales.drop(revenue_outliers_indecies)

In [40]:
# After filling missing values and removing outliers updating old dataset to cleaned dataset
df_clean_sales.to_csv('cleaned_sales_transaction.csv', index=False)

In [45]:
# Filling missing values using mode
df_customers['Industry'].fillna(df_customers['Industry'].mode()[0],inplace=True)

In [50]:
# Standardize Size entries to 'Large' 'Medium' or 'Small'
df_customers['CompanySize'] = df_customers['CompanySize'].str.strip().str.lower() 
df_customers['CompanySize'] = df_customers['CompanySize'].replace(
    {
        'L':'Large','large':'Large', 'l' : 'Large', 'LARGE': 'Large', 
        'M': 'Medium', 'm': 'Medium' , 'medium': 'Medium', 'MEDIUM': 'Medium', 
        's': 'Small' , 'S': 'Small', 'small': 'Small', 'SMALL': 'Small'
    }
)

In [51]:
# Checking Values changes from 'L':'Large','large':'Large', 'l' : 'Large', 'LARGE': 'Large', to only Large vice versa
df_customers.head()

,CustomerID,Industry,CompanySize,ChurnStatus
0,CUST66250,Healthcare,Medium,N
1,CUST89177,Finance,Medium,N
2,CUST47479,Retail,Medium,N
3,CUST42619,Retail,Large,N
4,CUST14638,Finance,Small,N


In [ ]:
# Updating old dataset to cleaned dataset
df_customers.to_csv('cleaned_customer_data.csv', index=False)

In [56]:
# Updating old dataset to cleaned dataset
df_cleaned_sales = pd.read_csv('cleaned_sales_transaction.csv')

In [59]:
# Changing datatype of Date from object to datetime
df_cleaned_sales['Date'] = pd.to_datetime(df_cleaned_sales['Date'], errors='coerce')

In [5]:
# Reading the cleaning dataset for further analysis
df_cleaned_transaction = pd.read_csv('cleaned_sales_transaction.csv')
df_cleaned_customer = pd.read_csv('cleaned_customer_data.csv')
df_cleaned_region = pd.read_csv('Region Metadata.csv')

In [9]:
# importing sqlite3 and making connection for database
import sqlite3, csv 
con = sqlite3.connect('Sales.db')
curr = con.cursor()

In [ ]:
# Install pretty table library for to show tables when executing sql query
!pip install ipython-sql pretty
import prettytable
prettytable.DEFAULT = 'DEFAULT'

In [11]:
#  loading the database
%load_ext sql

In [12]:
# Creating Connection to Database
%sql sqlite:///Sales.db

In [13]:
# Reading cleaned csv to sql and assigning name for table like Sales_Data
df_cleaned_transaction.to_sql('Sales_Data',con,if_exists='replace',index=False)

19507

In [14]:
# Reading cleaned csv to sql and assigning name for table like Customer_Data
df_cleaned_customer.to_sql('Customer_Data',con,if_exists='replace',index=False)

5000

In [15]:
# Reading cleaned csv to sql and assigning name for table like region_data
df_cleaned_region.to_sql('Region_Data',con,if_exists='replace',index=False)

10

In [ ]:
# Checking cleaned data is ok
df_cleaned_transaction.head()

In [ ]:
# Checking cleaned data is ok
df_cleaned_customer.head()

In [ ]:
# Checking cleaned data is ok
df_cleaned_region.head()

In [25]:
# executing sql query for sales data by region and assigning to variable
sales_by_region = %sql select c.CustomerID,  s.Region,sum(s.Revenue) as Total_Sales_by_Region from Customer_Data c join Sales_Data s on c.CustomerID = s.CustomerID group by Region

 * sqlite:///Sales.db
Done.


In [ ]:
# Checking the sales data by region
sales_by_region

In [ ]:
# Creating Bar-Chart Visualization to see the relation-ship between region and sales

import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
from matplotlib.ticker import FuncFormatter

plt.figure(figsize=(10,6)) 
ax = sns.barplot(data = sales_by_region.DataFrame(),x = 'Region', y='Total_Sales_by_Region',palette='Set2', edgecolor='black')

formatter = FuncFormatter(lambda x, _: f'{int(x):,}')
ax.yaxis.set_major_formatter(formatter)

for bar in plt.gca().patches:
    plt.annotate(format(bar.get_height()), 
                (bar.get_x()+bar.get_width()/2, bar.get_height()),
                 ha= 'center',va= 'bottom'
                )

plt.xticks(rotation=30)
plt.title('Total Sales by Region')
plt.tight_layout()
plt.show()

In [45]:
# Converting Region Data to csv for powerbi dashboard 
df_sales_by_region = sales_by_region.DataFrame()

 * sqlite:///Sales.db
Done.


In [44]:
# Converting Region Data to csv for powerbi dashboard 
df_sales_by_region.to_csv('sales_by_region.csv',index= False)

In [54]:
#sql query for product-tier to see how many product falls in each region
%%sql  count_of_productTier_by_region << 
WITH region_summary AS (
    SELECT 
        s.Region,
        SUM(CASE WHEN s.ProductTier = 'Enterprise' THEN 1 ELSE 0 END) as Enterprise,
        SUM(CASE WHEN s.ProductTier = 'Pro' THEN 1 ELSE 0 END) as Pro,
        SUM(CASE WHEN s.ProductTier = 'Basic' THEN 1 ELSE 0 END) as Basic
    FROM Customer_Data c 
    JOIN Sales_Data s ON c.CustomerID = s.CustomerID 
    GROUP BY s.Region
)
SELECT 
    Region,
    Enterprise,
    Pro,
    Basic
FROM region_summary
ORDER BY Region;

 * sqlite:///Sales.db
Done.
Returning data to local variable count_of_productTier_by_region


In [46]:
# just testing the query to see how i create logic for product-tier to see how many product falls in each region
test = %sql select s.Region from Customer_Data c join Sales_Data s on c.CustomerID = s.CustomerID group by s.Region

 * sqlite:///Sales.db
Done.


In [ ]:
# just testing the query to see how i create logic for product-tier to see how many product falls in each region
test

In [48]:
# just testing the query to see how i create logic for product-tier to see how many product falls in each region
test = %sql select count(s.ProductTier) from  Customer_Data c join Sales_Data s on c.CustomerID = s.CustomerID where s.ProductTier = 'Enterprise' group by s.Region

 * sqlite:///Sales.db
Done.


In [ ]:
# just testing the query to see how i create logic for product-tier to see how many product falls in each region
test

In [56]:
# Converting product-tier summary to csv for powerbi dashboard 
Region_Summary = count_of_productTier_by_region.DataFrame()

In [ ]:
# Checking Everthing works fine for product-tier to see how many product falls in each region
Region_Summary

In [84]:
# Checking Avg-revenue by region over-time to see the trend
%%sql Monthly_avg_revenue << 
SELECT 
    s.Region,
    strftime('%Y', s.Date) as Year,
    strftime('%m', s.Date) as Month,
    round(AVG(s.Revenue),2) as Avg_Revenue,
    COUNT(*) as Transaction_Count,
    SUM(s.Revenue) as Total_Revenue
FROM Customer_Data c 
JOIN Sales_Data s ON c.CustomerID = s.CustomerID 
GROUP BY s.Region, strftime('%Y', s.Date), strftime('%m', s.Date)
ORDER BY s.Region, Year, Month;

 * sqlite:///Sales.db
Done.
Returning data to local variable Monthly_avg_revenue


In [86]:
# Converting Avg-revenue data to DataFrame to csv for powerbi dashboard 
df_monthly_revenue = Monthly_avg_revenue.DataFrame()

In [87]:
# Converting Avg_Revenue data to csv for powerbi dashboard 
df_monthly_revenue.to_csv('avg_revenue_by_region.csv', index = False)

In [ ]:
# Checking the data is working fine
df_monthly_revenue

In [31]:
# Checking the Churn_status rate by region like how many customers are there in this how many are churned and active than their percentage
%%sql Churn_Status << 
SELECT 
    s.Region,
    COUNT(*) as total_customers,
    SUM(CASE WHEN c.ChurnStatus = 'Y' THEN 1 ELSE 0 END) as churned_customers,
    SUM(CASE WHEN c.ChurnStatus = 'N' THEN 1 ELSE 0 END) as active_customers,
    ROUND(
        (SUM(CASE WHEN c.ChurnStatus = 'Y' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 
        2
    ) as churn_rate_percentage
FROM 
    sales_data s
INNER JOIN 
    customer_data c ON s.CustomerID = c.CustomerID
GROUP BY 
    s.Region
ORDER BY 
    churn_rate_percentage DESC;

 * sqlite:///Sales.db
Done.
Returning data to local variable Churn_Status


In [ ]:
# Checking the data is working fine
Churn_Status

In [25]:
# Converting Churn_Status variable to DataFrame so i can convert into csv for visualiztion in power-bi
df_Churn_Status = Churn_Status.DataFrame()

In [29]:
# Converting Churn_Status data to csv for powerbi dashboard 
df_Churn_Status.to_csv('Churn_Status.csv', index=False)